# Train smolVLA
___

In [2]:
!cd .. && git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

fatal: destination path 'lerobot' already exists and is not an empty directory.
Channels:
 - conda-forge
 - nodefaults
 - nvidia
 - pytorch
 - defaults
Platform: linux-64
Obtaining file:///home/vorart/Downloads/Telegram%20Desktop/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.3.4-0.editable-py3-none-any.whl size=15738 sha256=bf12b8b42f797b5c4eaeb5043d98497fd70cd82a020a01bde2db4861ac39e7d2
  Stored in directory: /tmp/pip-ephem-wheel-cache-izwiimn2/wheels/cb/ec/e0/2b7466c2190a6a2942cfaaac396c27a9acc02af83943a7a84c
Successfully built lerobot
  Attempting uninstall: lerobot
    Found existing installation: lerobot 0.3.4
    Uninstalling lerobot-0.3.4:
      Successfully uninstalled lerobot-0.3.4


In [ ]:
!cd lerobot && pip install -e ".[smolvla]"

In [14]:
import sys, torch, platform, lerobot
print("PY:", sys.executable)
print("Torch:", torch.__version__, "| CUDA avail:", torch.cuda.is_available())
print("OS:", platform.platform())
print("Lerobot:", lerobot.__version__)

PY: /home/vorart/workspace/libs/miniforge3/bin/python
Torch: 2.7.1+cu126 | CUDA avail: True
OS: Linux-6.8.0-79-generic-x86_64-with-glibc2.39
Lerobot: 0.3.4


## Configs

In [ ]:
from pathlib import Path
import shutil, subprocess, sys, torch

# Paths
HF_USER   = "VorArt"
DATASET   = "hermesbot_dataset"
MODEL_NAME = "smolvla_test_finetune"

# IMPORTANT: must point to the dataset directory that contains meta/info.json
DATASET_DIR = Path.home() / f"workspace/robotics/Hermes/hf_home/lerobot/{HF_USER}/{DATASET}"

# Training output
OUT_DIR   = Path.home() / "workspace/robotics/Hermes/hermesbot-smolvla/model/train" / MODEL_NAME

# Train settings
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE  = 8
STEPS       = 1000
WANDB       = False   # flip True if you want W&B

print("DATASET_DIR:", DATASET_DIR)
print("OUT_DIR:", OUT_DIR)

args = [
    "--policy.path=lerobot/smolvla_base",
    f"--policy.repo_id={HF_USER}/{MODEL_NAME}",
    f"--policy.device={DEVICE}",
    f"--dataset.repo_id={HF_USER}/{DATASET}",
    f"--dataset.root={str(DATASET_DIR)}",
    f"--batch_size={BATCH_SIZE}",
    f"--steps={STEPS}",
    f"--output_dir={str(OUT_DIR)}",
    f"--job_name={MODEL_NAME}",
    f"--wandb.enable={'true' if WANDB else 'false'}",

    # Optimizer config
    "--optimizer.type=adamw",
    "--optimizer.lr=0.0001",
    "--optimizer.weight_decay=1e-10",
    "--optimizer.grad_clip_norm=10.0",

    # Scheduler config
    "--scheduler.type=cosine_decay_with_warmup",
    "--scheduler.num_warmup_steps=1000",
    "--scheduler.num_decay_steps=20000",
    "--scheduler.peak_lr=0.0001",
    "--scheduler.decay_lr=2.5e-6",
]

## Check dataset

In [17]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset

ds = LeRobotDataset(repo_id=f"{HF_USER}/{DATASET}", root=str(DATASET_DIR))
print("episodes:", ds.num_episodes, "frames:", ds.num_frames, "fps:", ds.fps)
print("cameras:", [k for k in ds.features if k.startswith("observation.images.")])

s = ds[0]
print({k: tuple(v.shape) for k,v in s.items()
       if k.startswith("observation.images.") or k in ("observation.state","action")})


episodes: 55 frames: 16064 fps: 15
cameras: ['observation.images.wrist', 'observation.images.base', 'observation.images.side']
{'action': (6,), 'observation.state': (6,), 'observation.images.wrist': (3, 480, 640), 'observation.images.base': (3, 480, 640), 'observation.images.side': (3, 480, 640)}


## Run traning

In [9]:
cmd = ["lerobot-train", *args] if shutil.which("lerobot-train") \
      else [sys.executable, "-m", "lerobot.scripts.train", *args]

print("RUN:", " ".join(cmd))
subprocess.run(cmd, check=True)

RUN: lerobot-train --policy.path=lerobot/smolvla_base --policy.repo_id=VorArt/smolvla_test_finetune --policy.device=cuda --dataset.repo_id=VorArt/hermesbot_dataset --dataset.root=/home/vorart/workspace/robotics/Hermes/hf_home/lerobot/VorArt/hermesbot_dataset --batch_size=8 --steps=1000 --output_dir=/home/vorart/workspace/robotics/Hermes/hermesbot-smolvla/model/train/smolvla_test_finetune --job_name=smolvla_test_finetune --wandb.enable=false --optimizer.type=adamw --optimizer.lr=0.0001 --optimizer.weight_decay=1e-10 --optimizer.grad_clip_norm=10.0 --scheduler.type=cosine_decay_with_warmup --scheduler.num_warmup_steps=1000 --scheduler.num_decay_steps=20000 --scheduler.peak_lr=0.0001 --scheduler.decay_lr=2.5e-6


/home/vorart/workspace/libs/miniforge3/lib/python3.12/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment FetchSlide-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/vorart/workspace/libs/miniforge3/lib/python3.12/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment FetchSlide-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/vorart/workspace/libs/miniforge3/lib/python3.12/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment FetchPickAndPlace-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/vorart/workspace/libs/miniforge3/lib/python3.12/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment FetchPickAndPlace-v3 already in registry.
  logger.warn(f"Overriding environ

Loading  HuggingFaceTB/SmolVLM2-500M-Video-Instruct weights ...


INFO 2025-09-13 21:04:48 odeling.py:1004 We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Reducing the number of VLM layers to 16 ...
[standardise_state_dict] 'normalize_inputs.buffer_observation_state.mean'  ←  ['normalize_inputs.so100-red_buffer_observation_state.mean', 'normalize_inputs.so100_buffer_observation_state.mean']
[standardise_state_dict] 'normalize_inputs.buffer_observation_state.std'  ←  ['normalize_inputs.so100-red_buffer_observation_state.std', 'normalize_inputs.so100_buffer_observation_state.std']
[standardise_state_dict] 'normalize_targets.buffer_action.mean'  ←  ['normalize_targets.so100-red_buffer_action.mean', 'normalize_targets.so100_buffer_action.mean']
[standardise_state_dict] 'normalize_targets.buffer_action.std'  ←  ['normalize_targets.so100-red_buffer_action.std', 'normalize_targets.so100_buffer_action.std']
[standardise_state_dict] 'unnormalize_outputs.buffer_action.mean'  ←  ['unnormalize_outputs.so100-red_buffer_action.mean', 'unnormalize_outputs.so100_buffer_action.mean']
[standardise_state_dict] 'unnormalize_outputs.buffer_action.std'  ←  ['

INFO 2025-09-13 21:04:55 ts/train.py:144 Creating optimizer and scheduler
INFO 2025-09-13 21:04:55 ts/train.py:156 Output dir: /home/vorart/workspace/robotics/Hermes/hermesbot-smolvla/model/train/smolvla_test_finetune
INFO 2025-09-13 21:04:55 ts/train.py:159 cfg.steps=1000 (1K)
INFO 2025-09-13 21:04:55 ts/train.py:160 dataset.num_frames=16064 (16K)
INFO 2025-09-13 21:04:55 ts/train.py:161 dataset.num_episodes=55
INFO 2025-09-13 21:04:55 ts/train.py:162 num_learnable_params=99880992 (100M)
INFO 2025-09-13 21:04:55 ts/train.py:163 num_total_params=450046212 (450M)
INFO 2025-09-13 21:04:55 ts/train.py:202 Start offline training on a fixed dataset
INFO 2025-09-13 21:05:57 ts/train.py:232 step:200 smpl:2K ep:5 epch:0.10 loss:0.118 grdn:1.384 lr:1.0e-05 updt_s:0.301 data_s:0.006
INFO 2025-09-13 21:07:03 ts/train.py:232 step:400 smpl:3K ep:11 epch:0.20 loss:0.076 grdn:1.195 lr:3.0e-05 updt_s:0.330 data_s:0.000
INFO 2025-09-13 21:08:22 ts/train.py:232 step:600 smpl:5K ep:16 epch:0.30 loss:0.06

CompletedProcess(args=['lerobot-train', '--policy.path=lerobot/smolvla_base', '--policy.repo_id=VorArt/smolvla_test_finetune', '--policy.device=cuda', '--dataset.repo_id=VorArt/hermesbot_dataset', '--dataset.root=/home/vorart/workspace/robotics/Hermes/hf_home/lerobot/VorArt/hermesbot_dataset', '--batch_size=8', '--steps=1000', '--output_dir=/home/vorart/workspace/robotics/Hermes/hermesbot-smolvla/model/train/smolvla_test_finetune', '--job_name=smolvla_test_finetune', '--wandb.enable=false', '--optimizer.type=adamw', '--optimizer.lr=0.0001', '--optimizer.weight_decay=1e-10', '--optimizer.grad_clip_norm=10.0', '--scheduler.type=cosine_decay_with_warmup', '--scheduler.num_warmup_steps=1000', '--scheduler.num_decay_steps=20000', '--scheduler.peak_lr=0.0001', '--scheduler.decay_lr=2.5e-6'], returncode=0)